In [11]:
import arxiv
import os
from datetime import datetime, timedelta

In [6]:
def download_papers_by_query(query, max_results=5, download_dir="./papers"):    
    os.makedirs(download_dir, exist_ok=True)
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=5,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    papers_downloaded = 0
    for result in client.results(search):
        try:
            # 下载论文
            result.download_pdf(dirpath=download_dir)
            papers_downloaded += 1
            
            print(f"已下载: {result.title}")
            print(f"作者: {', '.join(author.name for author in result.authors)}")
            print(f"发布时间: {result.published}")
            print(f"链接: {result.entry_id}")
            print(f"文件保存为: {result.get_short_id()}.pdf")
            print("-" * 80)
        except Exception as e:
            print(f"下载失败 {result.title}: {e}")
        
        print(f"下载完成! 共下载 {papers_downloaded} 篇论文")

In [7]:
download_papers_by_query("large language model")

已下载: From nuclear star clusters to Little Red Dots: black hole growth, mergers, and tidal disruptions
作者: Konstantinos Kritos, Joseph Silk
发布时间: 2025-10-24 17:59:58+00:00
链接: http://arxiv.org/abs/2510.21709v1
文件保存为: 2510.21709v1.pdf
--------------------------------------------------------------------------------
下载完成! 共下载 1 篇论文
已下载: Connecting Chemical Enrichment with Resolved Star Formation Histories
作者: Christopher T. Garling, Alex M. Garcia, Niusha Ahvazi, Nitya Kallivayalil, Kristen B. W. McQuinn, Robert Feldmann, Roger E. Cohen
发布时间: 2025-10-24 17:59:47+00:00
链接: http://arxiv.org/abs/2510.21707v1
文件保存为: 2510.21707v1.pdf
--------------------------------------------------------------------------------
下载完成! 共下载 2 篇论文
已下载: Equivariance by Contrast: Identifiable Equivariant Embeddings from Unlabeled Finite Group Actions
作者: Tobias Schmidt, Steffen Schneider, Matthias Bethge
发布时间: 2025-10-24 17:59:46+00:00
链接: http://arxiv.org/abs/2510.21706v1
文件保存为: 2510.21706v1.pdf
------------------

In [18]:
def advanced_paper_download(
    query, 
    max_results=10, 
    download_dir="./papers",
    categories=None,
    since_days=None
):
    """
    高级论文下载功能
    
    Args:
        query: 搜索词
        max_results: 最大结果数
        download_dir: 下载目录
        categories: 论文分类过滤
        since_days: 最近N天的论文
    """
    
    os.makedirs(download_dir, exist_ok=True)
    client = arxiv.Client()
    
    # 构建分类查询
    category_query = ""
    if categories:
        if isinstance(categories, str):
            categories = [categories]
        category_query = f" AND (cat:{" OR cat:".join(categories)})"
    
    # 构建时间查询
    date_query = ""
    if since_days:
        since_date = (datetime.now() - timedelta(days=since_days)).strftime("%Y%m%d")
        date_query = f" AND submittedDate:[{since_date}000000 TO {datetime.now().strftime('%Y%m%d')}235959]"
    
    # 完整查询
    full_query = f"({query}){category_query}{date_query}"
    
    print(full_query)

    search = arxiv.Search(
        query=full_query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    downloaded_papers = []

    for result in client.results(search):
        try:
            # 生成文件名：标题 + 作者
            authors_str = "_".join([author.name.split()[-1] for author in result.authors[:2]])
            filename = f"{result.title[:50]}_{authors_str}.pdf".replace("/", "-").replace("\\", "-")c
            filepath = os.path.join(download_dir, filename)

            # 下载论文
            print(filepath)
            result.download_pdf(filename=filepath)

            paper_info = {
                'title': result.title,
                'authors': [author.name for author in result.authors],
                'published': result.published,
                'summary': result.summary,
                'categories': result.categories,
                'pdf_url': result.pdf_url,
                'filepath': filepath
            }

            downloaded_papers.append(paper_info)

            print(f"✅ 下载: {result.title}")
            print(f"   作者: {', '.join(paper_info['authors'][:3])}")
            print(f"   分类: {', '.join(paper_info['categories'])}")
            print(f"   文件: {filename}")
            print()

        except Exception as e:
            print(f"❌ 下载失败 {result.title}: {e}")

    return downloaded_papers

In [19]:
papers = advanced_paper_download(
    query="large language model",
    max_results=5,
    categories="cs.CL",  # 计算机科学-计算语言学
    since_days=7,
    download_dir="./llm_papers"
)

print(f"下载完成! 共下载 {len(papers)} 篇论文")

(large language model) AND (cat:cs.CL) AND submittedDate:[20251020000000 TO 20251027235959]
./llm_papers\AstaBench: Rigorous Benchmarking of AI Agents with_Bragg_D'Arcy.pdf
✅ 下载: AstaBench: Rigorous Benchmarking of AI Agents with a Scientific Research Suite
   作者: Jonathan Bragg, Mike D'Arcy, Nishant Balepur
   分类: cs.AI, cs.CL
   文件: AstaBench: Rigorous Benchmarking of AI Agents with_Bragg_D'Arcy.pdf

./llm_papers\Few-Shot Knowledge Distillation of LLMs With Count_Hamman_Dissanayake.pdf
✅ 下载: Few-Shot Knowledge Distillation of LLMs With Counterfactual Explanations
   作者: Faisal Hamman, Pasan Dissanayake, Yanjun Fu
   分类: cs.LG, cs.AI, cs.CL, cs.CY, stat.ML
   文件: Few-Shot Knowledge Distillation of LLMs With Count_Hamman_Dissanayake.pdf

./llm_papers\The Universal Landscape of Human Reasoning_Chen_Liu.pdf
✅ 下载: The Universal Landscape of Human Reasoning
   作者: Qiguang Chen, Jinhao Liu, Libo Qin
   分类: cs.CL, cs.AI
   文件: The Universal Landscape of Human Reasoning_Chen_Liu.pdf

./llm_pa

In [ ]:
"././llm_papers\\AstaBench: Rigorous Benchmarking of AI Agents with_Bragg_D'Arcy.pdf"

In [1]:
data = {
  "data": [
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:基础信息\n索引关键词:数字人；介绍\n问题:介绍一下趣波，介绍一下数字人\n回复:苏州科技馆IP档案:\n中文名:趣波\n英文名:QuBoo(Qu(量子Quantum/探索Quest)+Boo(惊喜的拟声)\n性别:大暖男+天生童颜\n特征:好奇爱探索、理智乐观带点小傲娇,偶尔恶作剧却靠谱热衷科学实验和带领参观者游戏,集童真、智慧于一身。\n职务:首席体验官(虚拟辅导员)\n特长：勤奋，提供7x24小时不间断服务；聪明、爱学习爱探索，游馆时有问必答；\n数字人“趣波”是苏州科技馆打造的智慧客服大使。它集成了多项前沿技术：基于自动语音识别（ASR）与语音合成（TTS）技术，实现了流畅自然的真人化语音交互；通过高精度3D建模与表情动作驱动系统，呈现了生动逼真的视觉形象；结合大规模知识处理、自然语言理解与向量数据库技术，构筑了其强大的认知与问答能力。\n因此，“趣波”不仅能为您提供科技馆介绍、展品推荐、路线导览、票务与电影咨询等服务，还能作为智能助手，快速为您跳转对应的程序功能入口，让您的参观体验更智能、高效与便捷。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:展教活动\n索引关键词:活动\n问题:科技馆有哪些类型的活动？\n回复:苏州科技馆主要包括以下活动种类：依托展厅展品开发的教育课程与个性化讲解服务；在专属空间开展的科学教室系统培训；结合场馆与外部资源的研学旅行；可灵活使用的实验探究材料箱；通过线上平台传播的数字资源；覆盖权威竞赛与趣味活动的科技赛事以及融合讲座、表演、市集等形式的主题特色活动。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:基础信息\n索引关键词:科技馆；展品；介绍\n问题:苏州科技馆有什么重点展品？\n回复:（各展厅可列举2-3个与主题相关的展品）\n“微力无穷”展厅重点展项组有，“自然界的纳米高手”展项组，通过展项梦幻蝴蝶翅膀、蜻蜓复眼、荷叶迷宫等自然界已经存在的纳米级材料和结构，向观众揭示纳米的神奇之处；磁控治疗展项组，流动的磁液展示磁液这种材料的特性，而磁控医疗机器人则展示最新科研前线新型材料在意料中的应用。\n\n“探微历程”展厅重点展项有，“科学从哲学中走来”展项，生动形象得展示了科学观念是如何从哲学思辨里“生根发芽”的。“你能算出它的年龄吗？”展项将碳十四测年技术与苏州文旅相融合，沿着探索微观的脚步走过苏州的历史文脉。“欢迎来到平行宇宙”则邀请观众亲身参与由量子力学所引发的科幻亚文化的神奇世界。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:基础信息\n索引关键词:科技馆；介绍\n问题:苏州科技馆有别于其它科技馆的特色是什么？\n回复:苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明显差异化的、特色鲜明的科技馆。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:基础信息\n索引关键词:科技馆；介绍\n问题:介绍一下苏州科技馆\n回复:苏州科技馆位于苏州高新区狮山文化广场，总建筑面积约6.3万平方米，可布展面积约4万平方米。载体以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，包括“微中有机”、“无微不至”、“探微历程”、“微力无穷”、“察微知著”等10个常设展厅，以及3个临展厅、3个特效影院、7个展教实验空间等。苏州科技馆通过数字化沉浸式展陈体验、信息化场馆和智慧服务平台建设、受众与产品大数据分析等手段，着力于打造与传统科技馆具有明显差异化的、特色鲜明的新型科技馆。苏州科技馆运用数字化科技手段，围绕展教培训、科普活动、文创开发等业务，提升产品和服务品质，转化产业价值，力争建设成为面向长三角乃至全国青少年群体的具有数字化特色的科普阵地。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:展教活动\n索引关键词:培训\n问题:科技馆有哪些类型的培训？\n回复:1、素质拓展型：通过展厅活动、科学课程和研学服务，培养科学思维与动手能力，提升综合科学素养。2、学业辅助型：依托科学教室系列课程及实验材料箱，提供课后培训、巩固和拓展科学知识，助力学业水平提升。3、竞赛挑战型：围绕白名单等权威赛事开展培训，帮助学生获取竞赛成果，增强升学竞争力。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:场馆规则\n索引关键词:拍照；场馆规则\n问题:是否可以使用无人机或使用无人机拍摄？\n回复:禁止使用无人机\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:基础信息\n索引关键词:科技馆；介绍\n问题:苏州科技馆为什么以微生物为主题？\n回复:苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明显差异化的、特色鲜明的科技馆。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:展教活动\n索引关键词:赛事\n问题:科技馆有哪些类型的赛事？\n回复:苏州科技馆计划开展的赛事与征集活动主要分为三类：1、权威认证型赛事。承办教育部认可的“白名单\"竞赛，为参赛者提供官方报名通道。2、本土趣味型赛事。自主举办如“鸡头米科学奖”、“多爱地球三分钟\"等兼具科学性与趣味性的线下竟赛。3、创意征集型活动。通过线上线下渠道，向社会公众征集绘画、视频、创意设计等作品，并对优胜者予以奖励和宣传。\n"
    },
    {
      "documentId": "_sys_auto_gen_doc_id-9062699127954566761",
      "output": "问题类型:基础信息\n索引关键词:科技馆；介绍\n问题:介绍一下无微不至展厅\n回复:微生物虽小，却蕴藏着改变世界的巨大能量，在生态、健康、工业等领域掀起\"蝴蝶效应\"，并深刻影响着我们的日常生活与人类未来。无微不至展厅沿着“微生物的诞生与演化——微生物生态作用——微生物与人类——微生物与工农业——微生物与未来”的展示逻辑，依序从古老至今、从微观至宏观的叙事顺序，呈现展示微生物的生态功能及其在人类社会中的多重应用，最终展望未来技术的无限图景，引发观众对存在、共生与未来命题的思考。\n"
    }
  ],
  "query": "数字人“趣波”的特长有哪些？"
}

In [5]:
import re


data = data['data']
fully_matched = 0

output = []
for di in data:
    di = di['output']
    question_match = re.search(r'问题:(.*?)\n回复:', di)
    answer_match = re.search(r'回复:(.*)$', di)
    if question_match and answer_match:
        question = question_match.group(1)
        answer = answer_match.group(1)
        output.append({
                'ask': question,
                'answer': answer
            })
output = {
    # "data_str": '\n'.join([f"ask:{oi['ask']}\nanswer:{oi['answer']}\n" for oi in output]),
    "data_str": '\n'.join([f"{oi['ask']} {oi['answer']}\n" for oi in output]),
}

TypeError: list indices must be integers or slices, not str

In [6]:
output['data_str']

'科技馆有哪些类型的活动？ 苏州科技馆主要包括以下活动种类：依托展厅展品开发的教育课程与个性化讲解服务；在专属空间开展的科学教室系统培训；结合场馆与外部资源的研学旅行；可灵活使用的实验探究材料箱；通过线上平台传播的数字资源；覆盖权威竞赛与趣味活动的科技赛事以及融合讲座、表演、市集等形式的主题特色活动。\n\n苏州科技馆有别于其它科技馆的特色是什么？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明显差异化的、特色鲜明的科技馆。\n\n介绍一下苏州科技馆 苏州科技馆位于苏州高新区狮山文化广场，总建筑面积约6.3万平方米，可布展面积约4万平方米。载体以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，包括“微中有机”、“无微不至”、“探微历程”、“微力无穷”、“察微知著”等10个常设展厅，以及3个临展厅、3个特效影院、7个展教实验空间等。苏州科技馆通过数字化沉浸式展陈体验、信息化场馆和智慧服务平台建设、受众与产品大数据分析等手段，着力于打造与传统科技馆具有明显差异化的、特色鲜明的新型科技馆。苏州科技馆运用数字化科技手段，围绕展教培训、科普活动、文创开发等业务，提升产品和服务品质，转化产业价值，力争建设成为面向长三角乃至全国青少年群体的具有数字化特色的科普阵地。\n\n科技馆有哪些类型的培训？ 1、素质拓展型：通过展厅活动、科学课程和研学服务，培养科学思维与动手能力，提升综合科学素养。2、学业辅助型：依托科学教室系列课程及实验材料箱，提供课后培训、巩固和拓展科学知识，助力学业水平提升。3、竞赛挑战型：围绕白名单等权威赛事开展培训，帮助学生获取竞赛成果，增强升学竞争力。\n\n是否可以使用无人机或使用无人机拍摄？ 禁止使用无人机\n\n苏州科技馆为什么以微生物为主题？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明

In [7]:
di = "问题类型:基础信息\n索引关键词:数字人；介绍\n问题:介绍一下趣波，介绍一下数字人\n回复:苏州科技馆IP档案:\n中文名:趣波\n英文名:QuBoo(Qu(量子Quantum/探索Quest)+Boo(惊喜的拟声)\n性别:大暖男+天生童颜\n特征:好奇爱探索、理智乐观带点小傲娇,偶尔恶作剧却靠谱热衷科学实验和带领参观者游戏,集童真、智慧于一身。\n职务:首席体验官(虚拟辅导员)\n特长：勤奋，提供7x24小时不间断服务；聪明、爱学习爱探索，游馆时有问必答；\n数字人“趣波”是苏州科技馆打造的智慧客服大使。它集成了多项前沿技术：基于自动语音识别（ASR）与语音合成（TTS）技术，实现了流畅自然的真人化语音交互；通过高精度3D建模与表情动作驱动系统，呈现了生动逼真的视觉形象；结合大规模知识处理、自然语言理解与向量数据库技术，构筑了其强大的认知与问答能力。\n因此，“趣波”不仅能为您提供科技馆介绍、展品推荐、路线导览、票务与电影咨询等服务，还能作为智能助手，快速为您跳转对应的程序功能入口，让您的参观体验更智能、高效与便捷。\n"

In [12]:
question_match = re.search(r'问题:(.*?)\n回复:', di)
answer_match = re.search(r'回复:(.*)$', di)
if question_match and answer_match:
    question = question_match.group(1)
    answer = answer_match.group(1)
    output.append({
            'ask': question,
            'answer': answer
        })

In [13]:
output

{'data_str': '科技馆有哪些类型的活动？ 苏州科技馆主要包括以下活动种类：依托展厅展品开发的教育课程与个性化讲解服务；在专属空间开展的科学教室系统培训；结合场馆与外部资源的研学旅行；可灵活使用的实验探究材料箱；通过线上平台传播的数字资源；覆盖权威竞赛与趣味活动的科技赛事以及融合讲座、表演、市集等形式的主题特色活动。\n\n苏州科技馆有别于其它科技馆的特色是什么？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明显差异化的、特色鲜明的科技馆。\n\n介绍一下苏州科技馆 苏州科技馆位于苏州高新区狮山文化广场，总建筑面积约6.3万平方米，可布展面积约4万平方米。载体以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，包括“微中有机”、“无微不至”、“探微历程”、“微力无穷”、“察微知著”等10个常设展厅，以及3个临展厅、3个特效影院、7个展教实验空间等。苏州科技馆通过数字化沉浸式展陈体验、信息化场馆和智慧服务平台建设、受众与产品大数据分析等手段，着力于打造与传统科技馆具有明显差异化的、特色鲜明的新型科技馆。苏州科技馆运用数字化科技手段，围绕展教培训、科普活动、文创开发等业务，提升产品和服务品质，转化产业价值，力争建设成为面向长三角乃至全国青少年群体的具有数字化特色的科普阵地。\n\n科技馆有哪些类型的培训？ 1、素质拓展型：通过展厅活动、科学课程和研学服务，培养科学思维与动手能力，提升综合科学素养。2、学业辅助型：依托科学教室系列课程及实验材料箱，提供课后培训、巩固和拓展科学知识，助力学业水平提升。3、竞赛挑战型：围绕白名单等权威赛事开展培训，帮助学生获取竞赛成果，增强升学竞争力。\n\n是否可以使用无人机或使用无人机拍摄？ 禁止使用无人机\n\n苏州科技馆为什么以微生物为主题？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着

In [14]:
re.search(r'问题:(.*?)\n回复:', di)

<re.Match object; span=(23, 44), match='问题:介绍一下趣波，介绍一下数字人\n回复:'>

In [16]:
re.search(r'回复:(.*)$', di, re.DOTALL)

<re.Match object; span=(41, 458), match='回复:苏州科技馆IP档案:\n中文名:趣波\n英文名:QuBoo(Qu(量子Quantum/探索Q>